In [1]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [5]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [11]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [13]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, launch_site in launch_sites_df.iterrows():
    coord_launch = [launch_site['Lat'], launch_site['Long']]
    circle = folium.Circle(location = coord_launch, radius=100, color='#d35400', fill=True).add_child(folium.Popup(launch_site['Launch Site']))
    # Code for the marker
    marker = folium.map.Marker(coord_launch,
                               icon=DivIcon(
                                   icon_size=(20,20), 
                                   icon_anchor=(0,0),
                                   html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site['Launch Site'],
                               )
                              )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:1
- Are all launch sites in proximity to the Equator line2?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your find

1- They are not close to the equator, based on their latitudes and longitudes. However, it is clearly closer to the equator since the launch site are located at the south United States of America

2- Yes. It definitely looks at close proximity to the coast.ings.

In [16]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [18]:
marker_cluster = MarkerCluster()
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [20]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)   

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker([record['Lat'], record['Long']], 
                  icon=folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)
site_map

In [22]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [24]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [32]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
launch_site_lat = 28.563197
launch_site_long = -80.576820
coastline_lat = 28.56387
coastline_long = -80.56799
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [34]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

distance_marker =folium.Marker(
     [coastline_lat, coastline_long],
     icon=DivIcon(
         icon_size=(20,20),
         icon_anchor=(0,0),
         html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
         )
     )
site_map.add_child(distance_marker)

In [36]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
coordinates = [[launch_site_lat,launch_site_long],[coastline_lat,coastline_long]]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)


In [38]:
#Since we used CCAFS SLC-40 as the example. We will continue to use this specific launch site for the rest of the proximities. 

city_lat = 28.0668
city_long = -80.6095

railway_lat = 28.57139
railway_long = -80.58519

highway_lat = 28.56274
highway_long = -80.57072

distance_city = calculate_distance(launch_site_lat, launch_site_long, city_lat, city_long)
distance_railway = calculate_distance(launch_site_lat, launch_site_long, railway_lat, railway_long)
distance_highway = calculate_distance(launch_site_lat, launch_site_long, highway_lat, highway_long)

In [40]:
distance_marker1 =folium.Marker(
     [city_lat, city_long],
     icon=DivIcon(
         icon_size=(20,20),
         icon_anchor=(0,0),
         html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
         )
     )
site_map.add_child(distance_marker1)

distance_marker2 =folium.Marker(
     [railway_lat, railway_long],
     icon=DivIcon(
         icon_size=(20,20),
         icon_anchor=(0,0),
         html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
         )
     )
site_map.add_child(distance_marker2)

distance_marker3 =folium.Marker(
     [highway_lat, highway_long],
     icon=DivIcon(
         icon_size=(20,20),
         icon_anchor=(0,0),
         html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
         )
     )
site_map.add_child(distance_marker3)


In [46]:
coordinates1 = [[launch_site_lat,launch_site_long],[city_lat,city_long]]
lines1=folium.PolyLine(locations=coordinates1, weight=1)
site_map.add_child(lines1)

coordinates2 = [[launch_site_lat,launch_site_long],[railway_lat,railway_long]]
lines2 = folium.PolyLine(locations=coordinates2, weight=1)
site_map.add_child(lines2)

coordinates3 = [[launch_site_lat,launch_site_long],[highway_lat,highway_long]]
lines3 = folium.PolyLine(locations=coordinates3, weight=1)
site_map.add_child(lines3)

After you plot distance lines to the proximities, you can answer the following questions easily:1
- Are launch sites in close proximity to railways?2
- Are launch sites in close proximity to highways3?
- Are launch sites in close proximity to coastlin4e?
- Do launch sites keep certain distance away from cities?

Also please try to explain your fin

1- Yes. Due to quick movement of spares/parts.

2- Yes. For probably the same reason they are close to railways.

3- Yes. Due to the nature of rocket launches, it is always desirable to be near the coastline.

4- Yes. For safety reasons as well as public health regulations (e.g. noise pollution).dings.